# ⚙️ W05 — Hyperparameter Optimization

**Structure**:
- **Part A**: ML Branch — Strategy 1 (GA) vs Strategy 2 (Grid+KFold)
- **Part B**: DL Branch — Strategy 1 (GA) vs Strategy 2 (Grid+KFold)
- **Part C**: Final Comparison Table

**Author**: Fatima Khadija Benzine — February 2026

---
## 0. Setup

In [ ]:
import os
if not os.path.exists('/content/PhD-Project-'):
    !git clone https://github.com/f-khadija-benzine/PhD-Project-.git /content/PhD-Project-
!pip install xgboost -q
os.chdir('/content/PhD-Project-/src')

from google.colab import drive
drive.mount('/content/drive')

from pathlib import Path
from datetime import datetime
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import json, time, itertools

project_root = Path('/content/PhD-Project-')
TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M')
SAVE_DIR = f'/content/drive/MyDrive/PhD_results/W05_{TIMESTAMP}'
os.makedirs(SAVE_DIR, exist_ok=True)

from data_loader import MultiDatasetLoader
from preprocessing import DataNormalizer, create_sliding_windows, evaluate_per_unit
from bi_fusion import BIFusionPipeline, CONTINUOUS_BI_VARS
from feature_selection import BIAwareFeatureSelector
from feature_selection_aficv import AFICvFeatureSelector
from ml_branch import MLBranch, HybridPredictor
from attention import build_dual_attention_bilstm
from ga_optimizer import run_ml_ga, run_dl_ga
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error
import tensorflow as tf

plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

print(f"GPU: {tf.config.list_physical_devices('GPU')}")
print(f"Save: {SAVE_DIR}")
print("All imports ✓")

---
## 1. Data Preparation

In [ ]:
DATASET = 'FD001'
W = 30
PAD = False

loader = MultiDatasetLoader()
ds = loader.load_cmapss_dataset(DATASET)
meta_cols = ['unit', 'cycle', 'rul']
train_raw = ds['train'].copy()
test_raw = ds['test'].copy()
train_raw['rul'] = train_raw['rul'].clip(upper=125)
if 'rul' in test_raw.columns:
    test_raw['rul'] = test_raw['rul'].clip(upper=125)

sensor_cols = [c for c in train_raw.columns if c.startswith('sensor_')]
setting_cols = [c for c in train_raw.columns if c.startswith('setting_')]

norm = DataNormalizer(method='minmax')
train_norm = norm.fit_transform(train_raw, sensor_cols + setting_cols)
test_norm = norm.transform(test_raw)

fusion = BIFusionPipeline()
train_fused = fusion.fuse(train_norm, DATASET, split='train', encode_categoricals=True)
test_fused = fusion.fuse(test_norm, DATASET, split='test', encode_categoricals=True)
bi_cols = fusion.get_bi_columns(train_fused)
bi_cont = [c for c in CONTINUOUS_BI_VARS if c in train_fused.columns]
bi_norm = DataNormalizer(method='minmax')
train_fused = bi_norm.fit_transform(train_fused, bi_cont)
test_fused = bi_norm.transform(test_fused)

In [ ]:
# 3 feature selection datasets
X_train_dict, y_train_dict, train_df_dict = {}, {}, {}
X_test_dict, y_test_dict, test_df_dict = {}, {}, {}
feature_names_dict = {}

sel_corr = BIAwareFeatureSelector(variance_threshold=0.01, correlation_threshold=0.95)
fn_corr = sel_corr.select_features(data=train_fused, sensor_cols=sensor_cols,
    bi_cols=bi_cols, setting_cols=setting_cols, exclude_cols=meta_cols)
tr_corr = sel_corr.transform(train_fused, keep_cols=meta_cols)
te_corr = sel_corr.transform(test_fused, keep_cols=meta_cols)
X_train_dict['correlation'], y_train_dict['correlation'] = create_sliding_windows(tr_corr, W, fn_corr, 'rul', pad=PAD)
X_test_dict['correlation'], y_test_dict['correlation'] = create_sliding_windows(te_corr, W, fn_corr, 'rul', pad=PAD)
train_df_dict['correlation'], test_df_dict['correlation'] = tr_corr, te_corr
feature_names_dict['correlation'] = fn_corr

sel_aficv = AFICvFeatureSelector(base_learner='xgboost', n_folds=5, cumulative_threshold=0.90)
fn_aficv = sel_aficv.select_features_stratified(data=train_fused, sensor_cols=sensor_cols,
    bi_cols=bi_cols, setting_cols=setting_cols, target_col='rul', group_col='unit')
tr_aficv = sel_aficv.transform(train_fused, keep_cols=meta_cols)
te_aficv = sel_aficv.transform(test_fused, keep_cols=meta_cols)
X_train_dict['aficv'], y_train_dict['aficv'] = create_sliding_windows(tr_aficv, W, fn_aficv, 'rul', pad=PAD)
X_test_dict['aficv'], y_test_dict['aficv'] = create_sliding_windows(te_aficv, W, fn_aficv, 'rul', pad=PAD)
train_df_dict['aficv'], test_df_dict['aficv'] = tr_aficv, te_aficv
feature_names_dict['aficv'] = fn_aficv

fn_sensor = [f for f in fn_corr if f.startswith('sensor_') or f.startswith('setting_')]
tr_sensor = train_fused[meta_cols + fn_sensor].copy()
te_sensor = test_fused[meta_cols + fn_sensor].copy()
X_train_dict['sensor_only'], y_train_dict['sensor_only'] = create_sliding_windows(tr_sensor, W, fn_sensor, 'rul', pad=PAD)
X_test_dict['sensor_only'], y_test_dict['sensor_only'] = create_sliding_windows(te_sensor, W, fn_sensor, 'rul', pad=PAD)
train_df_dict['sensor_only'], test_df_dict['sensor_only'] = tr_sensor, te_sensor
feature_names_dict['sensor_only'] = fn_sensor

for k in X_train_dict:
    print(f"  {k:15s}: {X_train_dict[k].shape[2]:2d} feat, train={X_train_dict[k].shape[0]}, test={X_test_dict[k].shape[0]}")

In [ ]:
# K-Fold helpers
def get_unit_labels(df, window_size, pad):
    labels = []
    for u in sorted(df['unit'].unique()):
        T = len(df[df['unit'] == u])
        n_win = T if pad else max(T - (window_size - 1), 0)
        labels.extend([u] * n_win)
    return np.array(labels)

def rmse_per_unit(y_true, y_pred, unit_labels):
    preds_last, true_last = [], []
    for u in sorted(set(unit_labels)):
        mask = unit_labels == u
        if mask.sum() > 0:
            preds_last.append(y_pred[mask][-1])
            true_last.append(y_true[mask][-1])
    return np.sqrt(mean_squared_error(true_last, preds_last))

unit_labels_dict = {fs: get_unit_labels(train_df_dict[fs], W, PAD) for fs in X_train_dict}
ALL_RESULTS = []
print("Helpers ✓")

---
# ════════════════════════════════════════
# PART A — ML Branch (XGBoost)
# ════════════════════════════════════════

## A1. Strategy 1 — GA (20 pop × 30 gen)

In [ ]:
t0 = time.time()
ml_ga = run_ml_ga(
    X_train_dict=X_train_dict, y_train_dict=y_train_dict,
    train_df_dict=train_df_dict, feature_names_dict=feature_names_dict,
    window_size=W, pad=PAD, pop_size=20, n_generations=30, save_dir=SAVE_DIR)
ml_ga_time = time.time() - t0

In [ ]:
# A1 — Best combination
print(f"Best params: {ml_ga['best_params']}")
print(f"Val RMSE: {ml_ga['best_rmse']:.4f}")
print(f"Time: {ml_ga_time/60:.1f} min")

In [ ]:
# A1 — Retrain best model on full train
p = ml_ga['best_params']
fs = p['feature_selection']

ml_ga_model = MLBranch(model_type='xgboost', flatten_strategy=p['flatten_strategy'],
    n_estimators=p['n_estimators'], max_depth=p['max_depth'],
    learning_rate=p['learning_rate_xgb'], subsample=p.get('subsample', 0.8),
    colsample_bytree=p.get('colsample_bytree', 0.8),
    reg_alpha=p.get('reg_alpha', 0.1), reg_lambda=p.get('reg_lambda', 1.0))
ml_ga_model.fit(X_train_dict[fs], y_train_dict[fs], feature_names=feature_names_dict[fs])
y_pred = ml_ga_model.predict(X_test_dict[fs])

print(f"\n=== ML GA — TEST ({fs}) ===")
res = evaluate_per_unit(y_true=y_test_dict[fs], y_pred=y_pred,
    df=test_df_dict[fs], window_size=W, pad=PAD)

ALL_RESULTS.append({'Branch':'ML', 'Strategy':'GA', 'FeatureSel': fs,
    'Features': X_train_dict[fs].shape[2], 'Val_RMSE': ml_ga['best_rmse'],
    'Test_RMSE': res['rmse_last'], 'Test_Score': res['score_last'],
    'Time_min': round(ml_ga_time/60, 1)})

In [ ]:
# A1 — Save model + convergence
with open(f'{SAVE_DIR}/A1_ml_ga_best_{TIMESTAMP}.json', 'w') as f:
    json.dump({'params': ml_ga['best_params'], 'val_rmse': ml_ga['best_rmse'],
               'test_rmse': res['rmse_last'], 'test_score': res['score_last']}, f, indent=2)

h = ml_ga['ga'].get_history_df()
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(h['generation'], h['global_best'], 's-', label='Global best', linewidth=2)
ax.plot(h['generation'], h['mean_fitness'], '--', label='Gen mean', alpha=0.5)
ax.set_xlabel('Generation'); ax.set_ylabel('RMSE')
ax.set_title('A1 — ML GA Convergence'); ax.legend()
plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/A1_ml_ga_convergence_{TIMESTAMP}.png', dpi=150)
plt.show()

## A2. Strategy 2 — Grid Search + 5-Fold CV

In [ ]:
ML_GRID = {
    'n_estimators': [200, 300, 500],
    'max_depth': [4, 6, 8],
    'learning_rate_xgb': [0.05, 0.1],
    'flatten_strategy': ['statistics', 'flatten'],
    'feature_selection': ['correlation', 'aficv', 'sensor_only'],
}
N_FOLDS_ML = 5
ml_keys = list(ML_GRID.keys())
ml_combos = list(itertools.product(*[ML_GRID[k] for k in ml_keys]))
print(f"{len(ml_combos)} combos × {N_FOLDS_ML} folds = {len(ml_combos)*N_FOLDS_ML} fits")

In [ ]:
t0 = time.time()
ml_grid_all = []
best_cv = float('inf')
best_p = None

for i, combo in enumerate(ml_combos):
    params = dict(zip(ml_keys, combo))
    fs = params['feature_selection']
    X, y, groups = X_train_dict[fs], y_train_dict[fs], unit_labels_dict[fs]

    folds_rmse = []
    for tr_idx, va_idx in GroupKFold(n_splits=N_FOLDS_ML).split(X, y, groups):
        ml = MLBranch(model_type='xgboost', flatten_strategy=params['flatten_strategy'],
            n_estimators=params['n_estimators'], max_depth=params['max_depth'],
            learning_rate=params['learning_rate_xgb'], random_state=42)
        ml.fit(X[tr_idx], y[tr_idx], feature_names=feature_names_dict[fs], verbose=False)
        folds_rmse.append(rmse_per_unit(y[va_idx], ml.predict(X[va_idx]), groups[va_idx]))

    mean_r, std_r = np.mean(folds_rmse), np.std(folds_rmse)
    ml_grid_all.append({**params, 'mean_rmse': mean_r, 'std_rmse': std_r})

    if mean_r < best_cv:
        best_cv, best_p = mean_r, params.copy()

    if (i+1) % 10 == 0 or (i+1) == len(ml_combos):
        print(f"  [{i+1:3d}/{len(ml_combos)}] Best: {best_cv:.2f} | Cur: {mean_r:.2f}±{std_r:.2f} | {time.time()-t0:.0f}s")

    if (i+1) % 20 == 0:
        with open(f'{SAVE_DIR}/A2_ml_grid_checkpoint.json', 'w') as f:
            json.dump({'best_params': best_p, 'best_cv_rmse': best_cv}, f, indent=2)

ml_grid_time = time.time() - t0
print(f"\nDone — {ml_grid_time/60:.1f} min | Best CV: {best_cv:.4f}")
print(f"Best: {best_p}")

pd.DataFrame(ml_grid_all).to_csv(f'{SAVE_DIR}/A2_ml_grid_results_{TIMESTAMP}.csv', index=False)

In [ ]:
# A2 — Best combination
print(f"Best params: {best_p}")
print(f"CV RMSE: {best_cv:.4f} (5-fold)")

# Top 5
df_ml = pd.DataFrame(ml_grid_all).sort_values('mean_rmse')
print("\nTop 5:")
print(df_ml.head().to_string(index=False))

In [ ]:
# A2 — Retrain best model on full train
fs = best_p['feature_selection']

ml_grid_model = MLBranch(model_type='xgboost', flatten_strategy=best_p['flatten_strategy'],
    n_estimators=best_p['n_estimators'], max_depth=best_p['max_depth'],
    learning_rate=best_p['learning_rate_xgb'])
ml_grid_model.fit(X_train_dict[fs], y_train_dict[fs], feature_names=feature_names_dict[fs])
y_pred = ml_grid_model.predict(X_test_dict[fs])

print(f"\n=== ML Grid+5Fold — TEST ({fs}) ===")
res = evaluate_per_unit(y_true=y_test_dict[fs], y_pred=y_pred,
    df=test_df_dict[fs], window_size=W, pad=PAD)

ALL_RESULTS.append({'Branch':'ML', 'Strategy':'Grid+5Fold', 'FeatureSel': fs,
    'Features': X_train_dict[fs].shape[2], 'Val_RMSE': best_cv,
    'Test_RMSE': res['rmse_last'], 'Test_Score': res['score_last'],
    'Time_min': round(ml_grid_time/60, 1)})

In [ ]:
# A2 — Save
with open(f'{SAVE_DIR}/A2_ml_grid_best_{TIMESTAMP}.json', 'w') as f:
    json.dump({'params': best_p, 'cv_rmse': best_cv,
               'test_rmse': res['rmse_last'], 'test_score': res['score_last']}, f, indent=2)
print(f"Saved ✓")

---
# ════════════════════════════════════════
# PART B — DL Branch (BiLSTM + Dual Attention)
# ════════════════════════════════════════

## B1. Strategy 1 — GA (20 pop × 30 gen)

In [ ]:
t0 = time.time()
dl_ga = run_dl_ga(
    X_train_dict=X_train_dict, y_train_dict=y_train_dict,
    train_df_dict=train_df_dict, feature_names_dict=feature_names_dict,
    window_size=W, pad=PAD, pop_size=20, n_generations=30,
    max_epochs=50, save_dir=SAVE_DIR)
dl_ga_time = time.time() - t0

In [ ]:
# B1 — Best combination
print(f"Best params: {dl_ga['best_params']}")
print(f"Val RMSE: {dl_ga['best_rmse']:.4f}")
print(f"Time: {dl_ga_time/60:.1f} min")

In [ ]:
# B1 — Retrain best model
p = dl_ga['best_params']
fs = p['feature_selection']
n_feat = X_train_dict[fs].shape[2]

tf.keras.backend.clear_session()
model_ga, attn_ga = build_dual_attention_bilstm(
    window_size=W, n_features=n_feat,
    lstm_units=p['lstm_units'], feature_attention_dim=p['feature_attention_dim'],
    temporal_attention_dim=p['temporal_attention_dim'], dropout_rate=p['dropout_rate'],
    dense_units=p['dense_units'], learning_rate=p['learning_rate'])

model_ga.fit(X_train_dict[fs], y_train_dict[fs],
    epochs=100, batch_size=p['batch_size'], validation_split=0.2,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)],
    verbose=1)

y_pred = model_ga.predict(X_test_dict[fs], batch_size=256).flatten()

print(f"\n=== DL GA — TEST ({fs}) ===")
res = evaluate_per_unit(y_true=y_test_dict[fs], y_pred=y_pred,
    df=test_df_dict[fs], window_size=W, pad=PAD)

ALL_RESULTS.append({'Branch':'DL', 'Strategy':'GA', 'FeatureSel': fs,
    'Features': n_feat, 'Val_RMSE': dl_ga['best_rmse'],
    'Test_RMSE': res['rmse_last'], 'Test_Score': res['score_last'],
    'Time_min': round(dl_ga_time/60, 1)})

In [ ]:
# B1 — Save model + convergence
model_ga.save(f'{SAVE_DIR}/B1_dl_ga_model_{fs}_{TIMESTAMP}.keras')
with open(f'{SAVE_DIR}/B1_dl_ga_best_{TIMESTAMP}.json', 'w') as f:
    json.dump({'params': {k:v for k,v in p.items()}, 'val_rmse': dl_ga['best_rmse'],
               'test_rmse': res['rmse_last'], 'test_score': res['score_last']}, f, indent=2)

h = dl_ga['ga'].get_history_df()
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(h['generation'], h['global_best'], 's-', label='Global best', linewidth=2)
ax.plot(h['generation'], h['mean_fitness'], '--', label='Gen mean', alpha=0.5)
ax.set_xlabel('Generation'); ax.set_ylabel('RMSE')
ax.set_title('B1 — DL GA Convergence'); ax.legend()
plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/B1_dl_ga_convergence_{TIMESTAMP}.png', dpi=150)
plt.show()

## B2. Strategy 2 — Grid Search + 3-Fold CV

In [ ]:
DL_GRID = {
    'lstm_units': [32, 64],
    'feature_attention_dim': [32, 64],
    'temporal_attention_dim': [64, 128],
    'dropout_rate': [0.3],
    'dense_units': [32, 64],
    'learning_rate': [0.0005, 0.001],
    'batch_size': [128],
    'feature_selection': ['correlation', 'aficv', 'sensor_only'],
}
DL_FOLDS = 3
MAX_EPOCHS = 50

dl_keys = list(DL_GRID.keys())
dl_combos = list(itertools.product(*[DL_GRID[k] for k in dl_keys]))
print(f"{len(dl_combos)} combos × {DL_FOLDS} folds = {len(dl_combos)*DL_FOLDS} fits")

In [ ]:
t0 = time.time()
dl_grid_all = []
best_cv_dl = float('inf')
best_p_dl = None

for i, combo in enumerate(dl_combos):
    params = dict(zip(dl_keys, combo))
    fs = params['feature_selection']
    X, y = X_train_dict[fs], y_train_dict[fs]
    groups = unit_labels_dict[fs]
    n_feat = X.shape[2]

    folds_rmse = []
    for fold, (tr_idx, va_idx) in enumerate(GroupKFold(n_splits=DL_FOLDS).split(X, y, groups)):
        tf.keras.backend.clear_session()
        model, _ = build_dual_attention_bilstm(
            window_size=W, n_features=n_feat,
            lstm_units=params['lstm_units'],
            feature_attention_dim=params['feature_attention_dim'],
            temporal_attention_dim=params['temporal_attention_dim'],
            dropout_rate=params['dropout_rate'],
            dense_units=params['dense_units'],
            learning_rate=params['learning_rate'])
        model.fit(X[tr_idx], y[tr_idx], epochs=MAX_EPOCHS,
            batch_size=params['batch_size'],
            validation_data=(X[va_idx], y[va_idx]),
            callbacks=[tf.keras.callbacks.EarlyStopping(
                monitor='val_loss', patience=5, restore_best_weights=True)],
            verbose=0)
        y_pv = model.predict(X[va_idx], batch_size=256, verbose=0).flatten()
        folds_rmse.append(rmse_per_unit(y[va_idx], y_pv, groups[va_idx]))
        del model
        tf.keras.backend.clear_session()

    mean_r, std_r = np.mean(folds_rmse), np.std(folds_rmse)
    dl_grid_all.append({**params, 'mean_rmse': mean_r, 'std_rmse': std_r})

    if mean_r < best_cv_dl:
        best_cv_dl, best_p_dl = mean_r, params.copy()

    print(f"  [{i+1:3d}/{len(dl_combos)}] {fs:12s} | CV: {mean_r:.2f}±{std_r:.2f} | "
          f"Best: {best_cv_dl:.2f} | {time.time()-t0:.0f}s")

    if (i+1) % 5 == 0:
        with open(f'{SAVE_DIR}/B2_dl_grid_checkpoint.json', 'w') as f:
            json.dump({'best_params': best_p_dl, 'best_cv_rmse': best_cv_dl,
                       'progress': f'{i+1}/{len(dl_combos)}'}, f, indent=2)

dl_grid_time = time.time() - t0
print(f"\nDone — {dl_grid_time/60:.1f} min | Best CV: {best_cv_dl:.4f}")
print(f"Best: {best_p_dl}")

pd.DataFrame(dl_grid_all).to_csv(f'{SAVE_DIR}/B2_dl_grid_results_{TIMESTAMP}.csv', index=False)

In [ ]:
# B2 — Best combination + Top 5
print(f"Best params: {best_p_dl}")
print(f"CV RMSE: {best_cv_dl:.4f} (3-fold)")

df_dl = pd.DataFrame(dl_grid_all).sort_values('mean_rmse')
print("\nTop 5:")
print(df_dl.head().to_string(index=False))

In [ ]:
# B2 — Retrain best model
fs = best_p_dl['feature_selection']
n_feat = X_train_dict[fs].shape[2]

tf.keras.backend.clear_session()
model_grid, attn_grid = build_dual_attention_bilstm(
    window_size=W, n_features=n_feat,
    lstm_units=best_p_dl['lstm_units'],
    feature_attention_dim=best_p_dl['feature_attention_dim'],
    temporal_attention_dim=best_p_dl['temporal_attention_dim'],
    dropout_rate=best_p_dl['dropout_rate'],
    dense_units=best_p_dl['dense_units'],
    learning_rate=best_p_dl['learning_rate'])

model_grid.fit(X_train_dict[fs], y_train_dict[fs],
    epochs=100, batch_size=best_p_dl['batch_size'], validation_split=0.2,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)],
    verbose=1)

y_pred = model_grid.predict(X_test_dict[fs], batch_size=256).flatten()

print(f"\n=== DL Grid+3Fold — TEST ({fs}) ===")
res = evaluate_per_unit(y_true=y_test_dict[fs], y_pred=y_pred,
    df=test_df_dict[fs], window_size=W, pad=PAD)

ALL_RESULTS.append({'Branch':'DL', 'Strategy':'Grid+3Fold', 'FeatureSel': fs,
    'Features': n_feat, 'Val_RMSE': best_cv_dl,
    'Test_RMSE': res['rmse_last'], 'Test_Score': res['score_last'],
    'Time_min': round(dl_grid_time/60, 1)})

In [ ]:
# B2 — Save model
model_grid.save(f'{SAVE_DIR}/B2_dl_grid_model_{fs}_{TIMESTAMP}.keras')
with open(f'{SAVE_DIR}/B2_dl_grid_best_{TIMESTAMP}.json', 'w') as f:
    json.dump({'params': best_p_dl, 'cv_rmse': best_cv_dl,
               'test_rmse': res['rmse_last'], 'test_score': res['score_last']}, f, indent=2)
print(f"Saved ✓")

---
# ════════════════════════════════════════
# PART C — Final Comparison
# ════════════════════════════════════════

In [ ]:
comparison = pd.DataFrame(ALL_RESULTS)

print(f"\n{'='*80}")
print(f"FINAL COMPARISON — {DATASET}")
print(f"{'='*80}")
print(comparison.to_string(index=False))

comparison.to_csv(f'{SAVE_DIR}/C_final_comparison_{TIMESTAMP}.csv', index=False)
print(f"\n✓ Saved to {SAVE_DIR}")

In [ ]:
# Visual comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

labels = [f"{r['Branch']}\n{r['Strategy']}" for _, r in comparison.iterrows()]
colors = ['#2196F3' if r['Strategy']=='GA' else '#4CAF50' for _, r in comparison.iterrows()]

axes[0].barh(labels, comparison['Test_RMSE'], color=colors)
axes[0].set_xlabel('Test RMSE')
axes[0].set_title('Test RMSE Comparison')
for i, v in enumerate(comparison['Test_RMSE']):
    axes[0].text(v + 0.1, i, f'{v:.2f}', va='center')

axes[1].barh(labels, comparison['Test_Score'], color=colors)
axes[1].set_xlabel('Test Score (NASA)')
axes[1].set_title('Test Score Comparison')
for i, v in enumerate(comparison['Test_Score']):
    axes[1].text(v + 5, i, f'{v:.0f}', va='center')

plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/C_comparison_chart_{TIMESTAMP}.png', dpi=150)
plt.show()

---
## Files saved in Drive:
```
PhD_results/W05_YYYYMMDD_HHMM/
    A1_ml_ga_best_*.json           # ML GA best params + results
    A1_ml_ga_convergence_*.png     # ML GA convergence plot
    A2_ml_grid_results_*.csv       # All ML grid combinations
    A2_ml_grid_best_*.json         # ML Grid best params + results
    B1_dl_ga_model_*.keras         # DL GA best model
    B1_dl_ga_best_*.json           # DL GA best params + results
    B1_dl_ga_convergence_*.png     # DL GA convergence plot
    B2_dl_grid_model_*.keras       # DL Grid best model
    B2_dl_grid_results_*.csv       # All DL grid combinations
    B2_dl_grid_best_*.json         # DL Grid best params + results
    C_final_comparison_*.csv       # Summary table
    C_comparison_chart_*.png       # Comparison bar chart
```